In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# 라이브러리, 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold

In [3]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

## train Data

In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

- 수치형 특성

In [7]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


- 범주형 특성
(Cabin은 너무 많아 추후에 전처리 작업후 살펴보자)

In [8]:
cat_col = ['HomePlanet','CryoSleep','Destination','VIP']

In [9]:
for i in cat_col:
    print(10*"_", f"col={i}", 10*"_")
    print(train[i].value_counts())
    

__________ col=HomePlanet __________
Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64
__________ col=CryoSleep __________
False    5439
True     3037
Name: CryoSleep, dtype: int64
__________ col=Destination __________
TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64
__________ col=VIP __________
False    8291
True      199
Name: VIP, dtype: int64


## test Data

In [10]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [12]:
test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

- 수치형 특성

In [13]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


# EDA

### 카테고리별 Transported 관계 시각화

In [14]:
train_fig = train.copy()

In [15]:
train_fig.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

- 시각화에 쓰지 않을 데이터 버리기

In [16]:
train_fig = train_fig.drop(['Cabin','Name'], axis = 1)

- 가족 찾기

In [17]:
train_fig['Group'] = train_fig['PassengerId'].apply(lambda x : str(x[:4])).astype(int)

In [18]:
train_fig['FamilySize'] = train_fig.groupby('Group')['Group'].transform('count')

In [19]:
train_fig.head(15)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Group,FamilySize
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,1,1
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,2,1
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,3,2
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,3,2
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,4,1
5,0005_01,Earth,False,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,True,5,1
6,0006_01,Earth,False,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,True,6,2
7,0006_02,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,True,6,2
8,0007_01,Earth,False,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,True,7,1
9,0008_01,Europa,True,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,True,8,3


가족 컬럼이 잘 만들어졌다.

그리고 이제 가족 컬럼을 만들기 위해 남겨놓았던 승객 id지우기

---
# 이 밑에 정리 X (월)

In [20]:
train_fig = train_fig.drop(['PassengerId'], axis = 1)

In [21]:
fig = px.bar(train_fig, x = 'HomePlanet' ,y = 'Transported')
fig

In [22]:

fig = go.Figure(
    go.Bar(x=train_fig['HomePlanet'], y=train_fig['Transported']))
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

극히 일부가 많은 돈을 사용했다.

In [23]:
'''fig = px.histogram(train_pay, x = Pay,
                  marginal = 'box',nbins = 100)
fig.show()'''

"fig = px.histogram(train_pay, x = Pay,\n                  marginal = 'box',nbins = 100)\nfig.show()"

- passenger id 나누기 -> 가족

In [24]:
t = train.copy()

In [25]:
t['Group'] = t['PassengerId'].apply(lambda x : str(x[:4])).astype(int)
test['Group'] = test['PassengerId'].apply(lambda x : str(x[:4])).astype(int)

In [26]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
 14  Group         8693 non-null   int64  
dtypes: bool(1), float64(6), int64(1), object(7)
memory usage: 959.4+ KB


In [27]:
t['Group'].head(10)

0    1
1    2
2    3
3    3
4    4
5    5
6    6
7    6
8    7
9    8
Name: Group, dtype: int64

In [28]:
t['FamilySize'] = t.groupby('Group')['Group'].transform('count')
test['FamilySize'] = test.groupby('Group')['Group'].transform('count')

In [29]:
t[['Group','FamilySize']].head(15)

,Group,FamilySize
0,1,1
1,2,1
2,3,2
3,3,2
4,4,1
5,5,1
6,6,2
7,6,2
8,7,1
9,8,3


In [30]:
t

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,FamilySize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,1,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,2,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,3,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,3,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,9276,1
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,9278,1
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,9279,1
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,9280,2


In [31]:
t.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
Group             0
FamilySize        0
dtype: int64

In [32]:
t.drop(['Name','Cabin','Group','PassengerId'], axis = 1, inplace= True)
test.drop(['Name','Cabin','Group','PassengerId'], axis = 1, inplace= True)

In [33]:
X = t.drop('Transported', axis = 1)
y = t['Transported']

In [34]:
X_train, Y_train, X_target, Y_target = train_test_split(X,y,random_state = 12,test_size = 0.33)

모델링

In [35]:
lgb_params = {
    'objective' : 'binary',
    'n_estimators' : 50,
    'learning_rate' : 0.08
}

In [36]:
lgb_predictions = 0
# lgb_predictions = pd.DataFrame(lgb_predictions)
lgb_scores = []
lgb_fimp =[]

In [37]:
t.columns

Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported',
       'FamilySize'],
      dtype='object')

In [38]:
LGBM_FEATURES = list(t.drop('Transported',axis = 1))
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state=12)

In [39]:
LGBM_FEATURES 

['HomePlanet',
 'CryoSleep',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'FamilySize']

In [40]:
t['HomePlanet'].fillna('Z',inplace = True)
label_cols= ["HomePlanet", "CryoSleep", "Destination" ,"VIP"]


In [41]:
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] = LabelEncoder().fit_transform(test[col])
    return train, test

In [42]:
from sklearn.preprocessing import LabelEncoder

In [43]:
t, test = label_encoder(t,test,label_cols)

In [44]:
t.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,FamilySize
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,2
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,2
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,1


In [45]:
from lightgbm import LGBMClassifier

In [46]:
# 타겟 컬럼을 뺀 훈련 데이터 와 타겟 데이터를 split 분류하여 train_idx와 valid_idx에 저장
for fold, (train_idx, valid_idx) in enumerate(skf.split(t[LGBM_FEATURES], t['Transported'])):
    print(10*"=", f"Fold={fold+1}", 10*"=")
    # 시작 시간 저장
    # start_time = time.time()
    
    # 위에서 훈련 / 테스트 별 분류된 인덱스 값을 이용해 훈련 / 테스트 데이터로 분류
    X_train, X_valid = t.iloc[train_idx][LGBM_FEATURES],t.iloc[valid_idx][LGBM_FEATURES]
    # Y_train, Y_valid = train[TARGET].iloc[train_idx],train[TARGET].iloc[valid_idx]
    Y_train, Y_valid = t.iloc[train_idx]['Transported'],t.iloc[valid_idx]['Transported']
    
    # 모델 학습 (파라미터)
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, Y_train, verbose = 0)
    
    # 정확도 측정해서 저장
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(Y_valid, preds_valid)
    lgb_scores.append(acc)
    
    
    # run_time = time.time() - start_time
    
    # 컬럼 별 특성 중요도 
    fim = pd.DataFrame(index=LGBM_FEATURES,
                      data=model.feature_importances_,
                      columns=[f'{fold}_importance'])
    lgb_fimp.append(fim)
    
    
    print(f"Fold={fold+1}, Accuracy score : {acc :2f}%")
    
    # 예측하고 저장
    test_preds = model.predict(test[LGBM_FEATURES]) 
    lgb_predictions += test_preds/5

print("")
print("Mean Accuracy : ", np.mean(lgb_scores))

========== Fold=1 ==========
Fold=1, Accuracy score : 0.814261%
========== Fold=2 ==========


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning:

'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning:

'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.



Fold=2, Accuracy score : 0.778608%
========== Fold=3 ==========
Fold=3, Accuracy score : 0.788384%
========== Fold=4 ==========


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning:

'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning:

'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.



Fold=4, Accuracy score : 0.789413%
========== Fold=5 ==========
Fold=5, Accuracy score : 0.802071%

Mean Accuracy :  0.7945476117843475


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning:

'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.

